 <h2 style="font-size: 24px;">Dogecoin Price Predicition</h2> By Jodabeni Dondeli

 <p style="font-size: 18px;">Data Preprocessing</p> For this Model we will be using Kaggle to obtain our dogecoin price data. 

In [ ]:
import pandas as pd
import kagglehub
import os

# Download latest version of the Doge dataset. 
path = kagglehub.dataset_download("lycheezz/historical-data-for-dogeusd-from-2017-to-present")
csv_file = os.path.join(path, 'Dogecoin-USD Historical Data.csv')



In [1]:
print("Hello World!")


Hello World!


In [2]:
1+1

2